### ### final5- 처리 사항
- 변수제거
> groupId,matchId,killplace,(killpoint, rankpoint, winpoint => Weighted),damagedealt,matchduration, numgroups, roadkills, teamkill,vehicle destroys, maxplace

- 이상치 제거
> matchType은 이상치 없음<br>
> 'assists_mean', 'boosts_mean','headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean','longestKill_mean', 'rideDistance_mean','swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','DBNOs_mean','revives_mean' => quantile 상위 0.003퍼센트 이상치 제거<br> - 아웃라이어 final_3 에 비해 완화
> 평균값이기 때문에 walkingdistance==0이고 winplaceperc >=0.4 은 제거(핵쟁이일 가능성)

- 데이터 변환
> Headshotkills => headshotkillsrate = headshotkill/kill<br>
> matchId, groupId 별로 평균값으로 컬럼 대체<br>
> match 평균 변수 추가<br>
> matchsize 변수 추가<br>
> 각_mean 변수 / 각_match_mean 변수 => 컬럼개수 줄이고, 해당 판에서 그 사람의 위치를 비율로 만들어줄 수 있음<br>
> 각_mean 변수 제거하지 않고 그대로 둔다.
> 각 match별 max, min 변수 추가

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
import sklearn.preprocessing
import gc

In [2]:
# dataframe display 옵션
pd.set_option('display.max_columns' , 60)
pd.set_option('display.width', None)

In [3]:
df_train = pd.read_csv('./data/train_V2.csv', dtype={
                    'assists': 'int16',
                    'boosts': 'int16',
                    'DBNOs': 'int16',
                    'headshotKills': 'int16',
                    'heals': 'int16',
                    'kills': 'int16',
                    'killStreaks': 'int16',
                    'longestKill': 'float32',
                    'revives': 'int16',
                    'rideDistance': 'float32',
                    'swimDistance': 'float32',
                    'vehicleDestroys': 'int16',
                    'walkDistance': 'float32',
                    'weaponsAquired': 'int16',
                    'winPlacePerc': 'float32'
                })

In [4]:
df_train = df_train[df_train['maxPlace'] > 1]

In [5]:
# 필요 없는 칼럼 제거
del_list = ['maxPlace','damageDealt','killPlace', 'killPoints',
            'matchDuration','numGroups', 'rankPoints','roadKills','teamKills','vehicleDestroys','winPoints']
for col in del_list :
    del df_train[col]

df_train_mean = df_train.groupby(['matchId','groupId']).mean()
df_train_max = df_train.groupby(['matchId','groupId']).max()
df_train_min = df_train.groupby(['matchId','groupId']).min()
df_train_size = df_train.groupby(['matchId']).size().reset_index(name='match_size')
df_train_match_mean = df_train.groupby(['matchId']).mean()

df_train = pd.merge(df_train, df_train_mean, suffixes=["","_mean"], how = 'left', on=['matchId','groupId'])
del df_train_mean

In [11]:
df_train_max.head()

Id  assists  boosts  DBNOs  \
matchId        groupId                                                  
0000a43bce5eec 18b16ec699d8b6  6703bc191cff0a        0       1      2   
               236ab9e9c081b9  e7b325d63d8393        0       0      2   
               3a6addfa0df938  fc62a751955351        0       0      0   
               4bf06994bd4c9a  e8a07358000dce        0       1      0   
               4d1bbbc19b9084  9db8cb6643530a        1       4      4   

                               headshotKills  heals  kills  killStreaks  \
matchId        groupId                                                    
0000a43bce5eec 18b16ec699d8b6              0      1      2            1   
               236ab9e9c081b9              0      0      0            0   
               3a6addfa0df938              0      0      0            0   
               4bf06994bd4c9a              0      0      0            0   
               4d1bbbc19b9084              1      2      5            2   

                               longestKill  matchType  revives  rideDistance  \
matchId        groupId                                                         
0000a43bce5eec 18b16ec699d8b6    12.550000  squad-fpp        0      0.000000   
               236ab9e9c081b9     0.000000  squad-fpp        0      0.000000   
               3a6addfa0df938     0.000000  squad-fpp        0      0.000000   
               4bf06994bd4c9a     0.000000  squad-fpp        0      0.000000   
               4d1bbbc19b9084    33.419998  squad-fpp        0    159.899994   

                               swimDistance  walkDistance  weaponsAcquired  \
matchId        groupId                                                       
0000a43bce5eec 18b16ec699d8b6         0.000    189.399994                1   
               236ab9e9c081b9         0.000     56.160000                1   
               3a6addfa0df938         0.000      9.636000                1   
               4bf06994bd4c9a         0.000    770.400024               12   
               4d1bbbc19b9084         6.545   3335.000000                4   

                               winPlacePerc  
matchId        groupId                       
0000a43bce5eec 18b16ec699d8b6        0.3333  
               236ab9e9c081b9        0.0370  
               3a6addfa0df938        0.0000  
               4bf06994bd4c9a        0.3704  
               4d1bbbc19b9084        1.0000

In [13]:
df_train = pd.merge(df_train,df_train_min, suffixes=["","_min"], how="left", on=['matchId'])
del df_train_min

MemoryError: 

In [14]:
df_train = pd.merge(df_train,df_train_max, suffixes=["","_max"], how="left", on=['matchId'])
del df_train_max

MemoryError: 

In [15]:
df_train = pd.merge(df_train,df_train_match_mean, suffixes=["","_match_mean"], how="left", on=['matchId'])
del df_train_match_mean

In [ ]:
df_train = pd.merge(df_train, df_train_size, how='left', on=['matchId'])
del df_train_size

In [ ]:
df_train_cvt = df_train.copy()

In [59]:


# 회귀분석을 하기 위해 문자열을 범주값으로 변환
matchTyp = {'squad-fpp': 3, 'duo': 2, 'solo-fpp': 1, 'squad': 3, 'duo-fpp': 2, 'solo': 1,
       'normal-squad-fpp': 4, 'crashfpp': 4, 'flaretpp': 4, 'normal-solo-fpp': 4,
       'flarefpp': 4, 'normal-duo-fpp': 4, 'normal-duo': 4, 'normal-squad': 4,
       'crashtpp': 4, 'normal-solo': 4 }

df_train_cvt['matchType'] = df_train_cvt['matchType'].replace(matchTyp)
#필요 없는 칼럼 제거
del_list = ['assists', 'boosts', 'DBNOs',
       'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
       'revives', 'rideDistance', 'swimDistance', 'walkDistance',
       'weaponsAcquired','winPlacePerc_match_mean','winPlacePerc_mean','winPlacePerc_min','winPlacePerc_max']

# # null값 확인 및 
# # inplace=False로 하면 기존 혹은 새로운 변수에 할당해야 하고, inplace = True 하면 해당변수에 적용됨
# # na가 포함된 행을 제거하는 것은 데이터 소실이 크기 때문에 inplace=False가 default
# df_train_cvt.dropna(inplace = True)
# print(df_train_cvt.isnull().any().any())

for col in del_list :
    del df_train_cvt[col]
    
# # null값 확인 및 
# # inplace=False로 하면 기존 혹은 새로운 변수에 할당해야 하고, inplace = True 하면 해당변수에 적용됨
# # na가 포함된 행을 제거하는 것은 데이터 소실이 크기 때문에 inplace=False가 default
# df_train_cvt.dropna(inplace = True)
# print(df_train_cvt.isnull().any().any())


### solo

In [65]:
solo_train_df = df_train_cvt[df_train_cvt['matchType']==1]

del solo_train_df['DBNOs_mean']
del solo_train_df['revives_mean']
del solo_train_df['DBNOs_match_mean']
del solo_train_df['revives_match_mean']
del solo_train_df['DBNOs_max']
del solo_train_df['DBNOs_min']
del solo_train_df['revives_max']
del solo_train_df['revives_min']
del solo_train_df['Id_min']
del solo_train_df['Id_max']
del solo_train_df['groupId_min']
del solo_train_df['groupId_max']
del solo_train_df['matchType_max']
del solo_train_df['matchType_min']

gc.collect()

# solo
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean',
       'assists_min', 'boosts_min', 'headshotKills_min',
       'heals_min', 'kills_min', 'killStreaks_min', 'longestKill_min',
     'rideDistance_min', 'swimDistance_min',
       'walkDistance_min', 'weaponsAcquired_min', 'assists_max', 'boosts_max',
       'headshotKills_max', 'heals_max', 'kills_max', 'killStreaks_max',
       'longestKill_max', 'rideDistance_max',
       'swimDistance_max', 'walkDistance_max', 'weaponsAcquired_max']
quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(solo_train_df[col],0.997),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    solo_train_df = solo_train_df[solo_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
solo_train_df.drop(index=solo_train_df[(solo_train_df['walkDistance_mean']==0)&(solo_train_df['winPlacePerc']>=0.4)].index,inplace=True)

# data 변환 - headshotkills
solo_train_df['headshotKillrate_mean'] = solo_train_df['headshotKills_mean']/solo_train_df['kills_mean']
solo_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)
solo_train_df['headshotKillrate_match_mean'] = solo_train_df['headshotKills_match_mean']/solo_train_df['kills_match_mean']
solo_train_df['headshotKillrate_match_mean'].fillna(value=0,inplace=True)

mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
 'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','headshotKillrate_mean']
mean_match_list = ['assists_match_mean',
 'boosts_match_mean','headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
 'rideDistance_match_mean', 'swimDistance_match_mean', 'walkDistance_match_mean', 'weaponsAcquired_match_mean','headshotKillrate_match_mean']
name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
 'rideDistance', 'swimDistance', 'walkDistance', 'weaponsAcquired','headshotKillrate']
print(len(mean_list))
print(len(mean_match_list))
print(len(name_list))

for i in range(len(mean_list)):
    mean = mean_list[i]
    match = mean_match_list[i]
    name = name_list[i]  
    solo_train_df[name+'_ratio'] = solo_train_df[mean]/(solo_train_df[match])
    solo_train_df[name+'_ratio'].fillna(value=0,inplace=True)
#     del solo_train_df[mean]
    del solo_train_df[match]
round(solo_train_df,4)

def change_column_order(df, col_name, index): # 컬럼위치 변경
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

# data 단위 변환
log_transform = ['longestKill_mean','rideDistance_mean','swimDistance_mean', 'walkDistance_mean',
                 'longestKill_min','rideDistance_min','swimDistance_min', 'walkDistance_min',
                'longestKill_max','rideDistance_max','swimDistance_max', 'walkDistance_max']
for col in log_transform :
    solo_train_df[col+'_log'] = np.log(solo_train_df[col]+1)
    solo_train_df = change_column_order(solo_train_df,col+'_log',list(solo_train_df.columns).index('match_size'))
    del solo_train_df[col]
    
# winPlacePerc 컬럼위치 변경
solo_train_df = change_column_order(solo_train_df,'winPlacePerc',len(list(solo_train_df.columns)))

solo_train_df.to_csv('./data/final5/final5_solo_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 1.0, 'boosts_mean': 9.0, 'headshotKills_mean': 4.0, 'heals_mean': 17.0, 'kills_mean': 9.0, 'killStreaks_mean': 2.0, 'longestKill_mean': 298.89999, 'rideDistance_mean': 8964.891, 'swimDistance_mean': 294.4891, 'walkDistance_mean': 4973.0, 'weaponsAcquired_mean': 12.0, 'assists_min': 0.0, 'boosts_min': 0.0, 'headshotKills_min': 0.0, 'heals_min': 0.0, 'kills_min': 0.0, 'killStreaks_min': 0.0, 'longestKill_min': 0.0, 'rideDistance_min': 0.0, 'swimDistance_min': 0.0, 'walkDistance_min': 18.42, 'weaponsAcquired_min': 1.0, 'assists_max': 3.0, 'boosts_max': 16.0, 'headshotKills_max': 12.0, 'heals_max': 45.0, 'kills_max': 19.0, 'killStreaks_max': 9.0, 'longestKill_max': 702.20001, 'rideDistance_max': 28050.0, 'swimDistance_max': 1117.0, 'walkDistance_max': 11350.0, 'weaponsAcquired_max': 23.0}
12
12
12


### duo

In [67]:
duo_train_df = df_train_cvt[df_train_cvt['matchType']==2]

del duo_train_df['Id_min']
del duo_train_df['Id_max']
del duo_train_df['groupId_min']
del duo_train_df['groupId_max']
del duo_train_df['matchType_max']
del duo_train_df['matchType_min']

#  duo 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean',
       'assists_min', 'boosts_min', 'headshotKills_min','DBNOs_min','DBNOs_max','revives_min','revives_max',
       'heals_min', 'kills_min', 'killStreaks_min', 'longestKill_min',
     'rideDistance_min', 'swimDistance_min',
       'walkDistance_min', 'weaponsAcquired_min', 'assists_max', 'boosts_max',
       'headshotKills_max', 'heals_max', 'kills_max', 'killStreaks_max',
       'longestKill_max', 'rideDistance_max',
       'swimDistance_max', 'walkDistance_max', 'weaponsAcquired_max']
quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(duo_train_df[col],0.997),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    duo_train_df = duo_train_df[duo_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
duo_train_df.drop(index=duo_train_df[(duo_train_df['walkDistance_mean']==0)&(duo_train_df['winPlacePerc']>=0.4)].index,inplace=True)
 
# data 변환 - headshotkills
duo_train_df['headshotKillrate_mean'] = duo_train_df['headshotKills_mean']/duo_train_df['kills_mean']
duo_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)
duo_train_df['headshotKillrate_match_mean'] = duo_train_df['headshotKills_match_mean']/duo_train_df['kills_match_mean']
duo_train_df['headshotKillrate_match_mean'].fillna(value=0,inplace=True)

# data 변환 - ration
mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
 'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','headshotKillrate_mean','DBNOs_mean','revives_mean']
mean_match_list = ['assists_match_mean',
 'boosts_match_mean','headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
 'rideDistance_match_mean', 'swimDistance_match_mean', 'walkDistance_match_mean', 'weaponsAcquired_match_mean',
                   'headshotKillrate_match_mean','DBNOs_match_mean','revives_match_mean']
name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
 'rideDistance', 'swimDistance', 'walkDistance', 'weaponsAcquired','headshotKillrate','DBNOs','revives']
print(len(mean_list))
print(len(mean_match_list))
print(len(name_list))

for i in range(len(mean_list)):
    mean = mean_list[i]
    match = mean_match_list[i]
    name = name_list[i]  
    duo_train_df[name+'_ratio'] = duo_train_df[mean]/(duo_train_df[match])
    duo_train_df[name+'_ratio'].fillna(value=0,inplace=True)
#     del duo_train_df[mean]
    del duo_train_df[match]
round(duo_train_df,4)

# data 단위 변환
log_transform = ['longestKill_mean','rideDistance_mean','swimDistance_mean', 'walkDistance_mean',
                 'longestKill_min','rideDistance_min','swimDistance_min', 'walkDistance_min',
                'longestKill_max','rideDistance_max','swimDistance_max', 'walkDistance_max']
for col in log_transform :
    duo_train_df[col+'_log'] = np.log(duo_train_df[col]+1)
    duo_train_df = change_column_order(duo_train_df,col+'_log',list(duo_train_df.columns).index('match_size'))
    del duo_train_df[col]

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
duo_train_df = change_column_order(duo_train_df,'winPlacePerc',len(list(duo_train_df.columns)))

duo_train_df.to_csv('./data/final5/final5_duo_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 2.5, 'boosts_mean': 8.0, 'headshotKills_mean': 2.5, 'heals_mean': 14.0, 'kills_mean': 7.0, 'killStreaks_mean': 2.0, 'longestKill_mean': 250.2, 'rideDistance_mean': 8486.0, 'swimDistance_mean': 232.84999, 'walkDistance_mean': 5017.5, 'weaponsAcquired_mean': 10.0, 'assists_min': 0.0, 'boosts_min': 0.0, 'headshotKills_min': 0.0, 'DBNOs_min': 0.0, 'DBNOs_max': 14.0, 'revives_min': 0.0, 'revives_max': 12.0, 'heals_min': 0.0, 'kills_min': 0.0, 'killStreaks_min': 0.0, 'longestKill_min': 0.0, 'rideDistance_min': 0.0, 'swimDistance_min': 0.0, 'walkDistance_min': 36.22, 'weaponsAcquired_min': 1.0, 'assists_max': 6.0, 'boosts_max': 14.0, 'headshotKills_max': 10.0, 'heals_max': 36.0, 'kills_max': 17.0, 'killStreaks_max': 6.0, 'longestKill_max': 736.59998, 'rideDistance_max': 17070.0, 'swimDistance_max': 924.79999, 'walkDistance_max': 10010.0, 'weaponsAcquired_max': 22.0}
14
14
14


### squad

In [68]:
squad_train_df = df_train_cvt[df_train_cvt['matchType']==3]

del squad_train_df['Id_min']
del squad_train_df['Id_max']
del squad_train_df['groupId_min']
del squad_train_df['groupId_max']
del squad_train_df['matchType_max']
del squad_train_df['matchType_min']

#  squad 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean',
       'assists_min', 'boosts_min', 'headshotKills_min','DBNOs_min','DBNOs_max','revives_min','revives_max',
       'heals_min', 'kills_min', 'killStreaks_min', 'longestKill_min',
     'rideDistance_min', 'swimDistance_min',
       'walkDistance_min', 'weaponsAcquired_min', 'assists_max', 'boosts_max',
       'headshotKills_max', 'heals_max', 'kills_max', 'killStreaks_max',
       'longestKill_max', 'rideDistance_max',
       'swimDistance_max', 'walkDistance_max', 'weaponsAcquired_max']
quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(squad_train_df[col],0.997),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    squad_train_df = squad_train_df[squad_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
squad_train_df.drop(index=squad_train_df[(squad_train_df['walkDistance_mean']==0)&(squad_train_df['winPlacePerc']>=0.4)].index,inplace=True)
 
# data 변환 - headshotkills
squad_train_df['headshotKillrate_mean'] = squad_train_df['headshotKills_mean']/squad_train_df['kills_mean']
squad_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)
squad_train_df['headshotKillrate_match_mean'] = squad_train_df['headshotKills_match_mean']/squad_train_df['kills_match_mean']
squad_train_df['headshotKillrate_match_mean'].fillna(value=0,inplace=True)

# data 변환 - ration
mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
 'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','headshotKillrate_mean','DBNOs_mean','revives_mean']
mean_match_list = ['assists_match_mean',
 'boosts_match_mean','headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
 'rideDistance_match_mean', 'swimDistance_match_mean', 'walkDistance_match_mean', 'weaponsAcquired_match_mean',
                   'headshotKillrate_match_mean','DBNOs_match_mean','revives_match_mean']
name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
 'rideDistance', 'swimDistance', 'walkDistance', 'weaponsAcquired','headshotKillrate','DBNOs','revives']
print(len(mean_list))
print(len(mean_match_list))
print(len(name_list))

for i in range(len(mean_list)):
    mean = mean_list[i]
    match = mean_match_list[i]
    name = name_list[i]  
    squad_train_df[name+'_ratio'] = squad_train_df[mean]/(squad_train_df[match])
    squad_train_df[name+'_ratio'].fillna(value=0,inplace=True)
#     del squad_train_df[mean]
    del squad_train_df[match]
round(squad_train_df,4)

# data 단위 변환
log_transform = ['longestKill_mean','rideDistance_mean','swimDistance_mean', 'walkDistance_mean',
                 'longestKill_min','rideDistance_min','swimDistance_min', 'walkDistance_min',
                'longestKill_max','rideDistance_max','swimDistance_max', 'walkDistance_max']
for col in log_transform :
    squad_train_df[col+'_log'] = np.log(squad_train_df[col]+1)
    squad_train_df = change_column_order(squad_train_df,col+'_log',list(squad_train_df.columns).index('match_size'))
    del squad_train_df[col]

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
squad_train_df = change_column_order(squad_train_df,'winPlacePerc',len(list(squad_train_df.columns)))

squad_train_df.to_csv('./data/final5/final5_squad_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 2.66667, 'boosts_mean': 6.75, 'headshotKills_mean': 2.0, 'heals_mean': 13.0, 'kills_mean': 6.0, 'killStreaks_mean': 2.33333, 'longestKill_mean': 229.7, 'rideDistance_mean': 8225.33301, 'swimDistance_mean': 195.08671, 'walkDistance_mean': 4954.25, 'weaponsAcquired_mean': 9.33333, 'assists_min': 0.0, 'boosts_min': 0.0, 'headshotKills_min': 0.0, 'DBNOs_min': 0.0, 'DBNOs_max': 17.0, 'revives_min': 0.0, 'revives_max': 8.0, 'heals_min': 0.0, 'kills_min': 0.0, 'killStreaks_min': 0.0, 'longestKill_min': 0.0, 'rideDistance_min': 0.0, 'swimDistance_min': 0.0, 'walkDistance_min': 43.68, 'weaponsAcquired_min': 1.0, 'assists_max': 7.0, 'boosts_max': 14.0, 'headshotKills_max': 11.0, 'heals_max': 37.0, 'kills_max': 18.0, 'killStreaks_max': 6.0, 'longestKill_max': 898.29999, 'rideDistance_max': 15900.0, 'swimDistance_max': 926.29999, 'walkDistance_max': 10120.0, 'weaponsAcquired_max': 21.0}
14
14
14


### other

In [69]:
other_train_df = df_train_cvt[df_train_cvt['matchType']==4]

del other_train_df['Id_min']
del other_train_df['Id_max']
del other_train_df['groupId_min']
del other_train_df['groupId_max']
del other_train_df['matchType_max']
del other_train_df['matchType_min']

#  squad 리스트
quantile_list = ['assists_mean', 'boosts_mean',
       'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean',
       'longestKill_mean', 'rideDistance_mean',
       'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean',
       'assists_min', 'boosts_min', 'headshotKills_min','DBNOs_min','DBNOs_max','revives_min','revives_max',
       'heals_min', 'kills_min', 'killStreaks_min', 'longestKill_min',
     'rideDistance_min', 'swimDistance_min',
       'walkDistance_min', 'weaponsAcquired_min', 'assists_max', 'boosts_max',
       'headshotKills_max', 'heals_max', 'kills_max', 'killStreaks_max',
       'longestKill_max', 'rideDistance_max',
       'swimDistance_max', 'walkDistance_max', 'weaponsAcquired_max']

quantile_dic = {}
for col in quantile_list:
    tmp_np = round(np.quantile(other_train_df[col],0.997),5)
    quantile_dic[col] = tmp_np
print(quantile_dic)

# 이상치 제거 1
for col in quantile_dic:
    other_train_df = other_train_df[other_train_df[col] <= quantile_dic[col]]
    
# 이상치 제거 2
other_train_df.drop(index=other_train_df[(other_train_df['walkDistance_mean']==0)&(other_train_df['winPlacePerc']>=0.4)].index,inplace=True)
 
# data 변환 - headshotkills
other_train_df['headshotKillrate_mean'] = other_train_df['headshotKills_mean']/other_train_df['kills_mean']
other_train_df['headshotKillrate_mean'].fillna(value=0,inplace=True)
other_train_df['headshotKillrate_match_mean'] = other_train_df['headshotKills_match_mean']/other_train_df['kills_match_mean']
other_train_df['headshotKillrate_match_mean'].fillna(value=0,inplace=True)

# data 변환 - ration
mean_list = ['assists_mean', 'boosts_mean', 'headshotKills_mean', 'heals_mean', 'kills_mean', 'killStreaks_mean', 'longestKill_mean',
 'rideDistance_mean', 'swimDistance_mean', 'walkDistance_mean', 'weaponsAcquired_mean','headshotKillrate_mean','DBNOs_mean','revives_mean']
mean_match_list = ['assists_match_mean',
 'boosts_match_mean','headshotKills_match_mean', 'heals_match_mean', 'kills_match_mean', 'killStreaks_match_mean', 'longestKill_match_mean',
 'rideDistance_match_mean', 'swimDistance_match_mean', 'walkDistance_match_mean', 'weaponsAcquired_match_mean',
                   'headshotKillrate_match_mean','DBNOs_match_mean','revives_match_mean']
name_list = ['assists', 'boosts', 'headshotKills', 'heals', 'kills', 'killStreaks', 'longestKill',
 'rideDistance', 'swimDistance', 'walkDistance', 'weaponsAcquired','headshotKillrate','DBNOs','revives']
print(len(mean_list))
print(len(mean_match_list))
print(len(name_list))

for i in range(len(mean_list)):
    mean = mean_list[i]
    match = mean_match_list[i]
    name = name_list[i]  
    other_train_df[name+'_ratio'] = other_train_df[mean]/(other_train_df[match])
    other_train_df[name+'_ratio'].fillna(value=0,inplace=True)
#     del other_train_df[mean]
    del other_train_df[match]
round(other_train_df,4)

# data 단위 변환
log_transform = ['longestKill_mean','rideDistance_mean','swimDistance_mean', 'walkDistance_mean',
                 'longestKill_min','rideDistance_min','swimDistance_min', 'walkDistance_min',
                'longestKill_max','rideDistance_max','swimDistance_max', 'walkDistance_max']
for col in log_transform :
    other_train_df[col+'_log'] = np.log(other_train_df[col]+1)
    other_train_df = change_column_order(other_train_df,col+'_log',list(other_train_df.columns).index('match_size'))
    del other_train_df[col]

# winPlacePerc 컬럼위치 변경
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
other_train_df = change_column_order(other_train_df,'winPlacePerc',len(list(other_train_df.columns)))

other_train_df.to_csv('./data/final5/final5_other_Data.csv',sep=',', encoding='utf-8', index=False)

{'assists_mean': 8.0, 'boosts_mean': 7.25, 'headshotKills_mean': 7.66667, 'heals_mean': 16.0, 'kills_mean': 25.0, 'killStreaks_mean': 3.625, 'longestKill_mean': 294.22501, 'rideDistance_mean': 10277.0, 'swimDistance_mean': 242.16251, 'walkDistance_mean': 5160.0, 'weaponsAcquired_mean': 49.0, 'assists_min': 1.0, 'boosts_min': 0.0, 'headshotKills_min': 1.0, 'DBNOs_min': 1.0, 'DBNOs_max': 39.0, 'revives_min': 0.0, 'revives_max': 15.0, 'heals_min': 0.0, 'kills_min': 6.0, 'killStreaks_min': 1.0, 'longestKill_min': 44.83, 'rideDistance_min': 767.0, 'swimDistance_min': 0.0, 'walkDistance_min': 1161.0, 'weaponsAcquired_min': 26.0, 'assists_max': 15.0, 'boosts_max': 15.0, 'headshotKills_max': 46.0, 'heals_max': 37.0, 'kills_max': 56.0, 'killStreaks_max': 14.0, 'longestKill_max': 1081.0, 'rideDistance_max': 14170.0, 'swimDistance_max': 1669.0, 'walkDistance_max': 8895.0, 'weaponsAcquired_max': 71.0}
14
14
14
